In [1]:
import tkinter as tk
from tkinter import messagebox, filedialog
import PyPDF2
from tkinterdnd2 import TkinterDnD, DND_FILES

class PDFMergerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Encadenar Archivos PDF")
        self.root.geometry("600x400")
        
        self.files = []  # Lista para almacenar los archivos PDF

        # Frame para los botones
        self.frame_buttons = tk.Frame(self.root)
        self.frame_buttons.grid(row=0, column=0, padx=10, pady=10, sticky="w")

        # Etiqueta para los botones
        self.label_buttons = tk.Label(self.frame_buttons, text="Acciones:", font=("Arial", 12))
        self.label_buttons.grid(row=0, column=0, padx=10, pady=5, sticky="w")

        # Botón para agregar archivos
        self.add_button = tk.Button(self.frame_buttons, text="Agregar Archivos", command=self.add_files)
        self.add_button.grid(row=1, column=0, padx=10, pady=5)

        # Botón para fusionar los PDFs
        self.merge_button = tk.Button(self.frame_buttons, text="Fusionar PDFs", command=self.merge_pdfs)
        self.merge_button.grid(row=1, column=1, padx=10, pady=5)

        # Frame para el Listbox y botones de reordenar
        self.frame_list = tk.Frame(self.root)
        self.frame_list.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")

        # Etiqueta para la lista de archivos
        self.label_list = tk.Label(self.frame_list, text="Archivos PDF a fusionar:", font=("Arial", 12))
        self.label_list.grid(row=0, column=0, padx=10, pady=5, sticky="w")

        # Lista de archivos (con capacidad de arrastrar y soltar)
        self.file_listbox = tk.Listbox(self.frame_list, selectmode=tk.SINGLE, width=50, height=10)
        self.file_listbox.grid(row=1, column=0, padx=10, pady=5)

        # Botones para mover los archivos hacia arriba y abajo
        self.move_up_button = tk.Button(self.frame_list, text="↑", command=self.move_up)
        self.move_up_button.grid(row=1, column=1, padx=10, pady=5, sticky="n")

        self.move_down_button = tk.Button(self.frame_list, text="↓", command=self.move_down)
        self.move_down_button.grid(row=1, column=1, padx=10, pady=5, sticky="s")

        # Configurar drag-and-drop
        self.file_listbox.drop_target_register(DND_FILES)
        self.file_listbox.dnd_bind('<<Drop>>', self.on_drop)

    def add_files(self):
        """Agrega archivos manualmente mediante un diálogo de archivo"""
        file_paths = filedialog.askopenfilenames(filetypes=[("Archivos PDF", "*.pdf")])
        
        if file_paths:
            for file_path in file_paths:
                if file_path not in self.files:
                    self.files.append(file_path)
                    self.file_listbox.insert(tk.END, file_path)
                else:
                    messagebox.showwarning("Advertencia", f"El archivo {file_path} ya está en la lista.")

    def on_drop(self, event):
        """Añade archivos arrastrados a la lista"""
        file_path = event.data
        if file_path.endswith(".pdf"):
            if file_path not in self.files:
                self.files.append(file_path)
                self.file_listbox.insert(tk.END, file_path)
            else:
                messagebox.showwarning("Advertencia", f"El archivo {file_path} ya está en la lista.")
        else:
            messagebox.showwarning("Advertencia", "Solo puedes agregar archivos PDF.")

    def move_up(self):
        """Mueve el archivo seleccionado hacia arriba"""
        selected_index = self.file_listbox.curselection()
        if selected_index:
            selected_index = selected_index[0]
            if selected_index > 0:
                # Intercambiar los elementos en la lista interna y en el Listbox
                self.files[selected_index], self.files[selected_index - 1] = self.files[selected_index - 1], self.files[selected_index]
                self.update_listbox()

    def move_down(self):
        """Mueve el archivo seleccionado hacia abajo"""
        selected_index = self.file_listbox.curselection()
        if selected_index:
            selected_index = selected_index[0]
            if selected_index < len(self.files) - 1:
                # Intercambiar los elementos en la lista interna y en el Listbox
                self.files[selected_index], self.files[selected_index + 1] = self.files[selected_index + 1], self.files[selected_index]
                self.update_listbox()

    def update_listbox(self):
        """Actualiza la visualización del Listbox con el nuevo orden"""
        self.file_listbox.delete(0, tk.END)
        for file_path in self.files:
            self.file_listbox.insert(tk.END, file_path)

    def merge_pdfs(self):
        """Fusiona los PDFs en un único archivo"""
        if not self.files:
            messagebox.showwarning("Advertencia", "No hay archivos PDF para fusionar.")
            return
        
        # Crear un objeto PDF merger
        pdf_merger = PyPDF2.PdfMerger()
        
        # Agregar los archivos PDF en el orden que están en la lista
        for file in self.files:
            pdf_merger.append(file)
        
        # Guardar el archivo final
        save_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
        
        if save_path:
            pdf_merger.write(save_path)
            messagebox.showinfo("Éxito", f"PDF fusionado guardado en {save_path}")
            self.files.clear()
            self.file_listbox.delete(0, tk.END)

# Inicializar la ventana de la aplicación
if __name__ == "__main__":
    root = TkinterDnD.Tk()  # Usamos TkinterDnD para soportar drag-and-drop
    app = PDFMergerApp(root)
    root.mainloop()


¡Claro! Vamos a mejorar el programa para hacerlo más intuitivo, con las siguientes modificaciones:

### 1. **Reordenar los archivos en el `Listbox`:**
   Vamos a añadir botones para mover los archivos hacia arriba y hacia abajo dentro del `Listbox`. Esto permitirá cambiar el orden de los archivos antes de fusionarlos.

### 2. **Interfaz más amigable:**
   - Añadiremos etiquetas y un diseño más ordenado para que la interfaz se vea más profesional.
   - Usaremos un diseño con `grid` para alinear mejor los elementos.

### Aquí está la versión mejorada del código:

```python
import tkinter as tk
from tkinter import messagebox, filedialog
import PyPDF2
from tkinterdnd2 import TkinterDnD, DND_FILES

class PDFMergerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Encadenar Archivos PDF")
        self.root.geometry("600x400")
        
        self.files = []  # Lista para almacenar los archivos PDF

        # Frame para los botones
        self.frame_buttons = tk.Frame(self.root)
        self.frame_buttons.grid(row=0, column=0, padx=10, pady=10, sticky="w")

        # Etiqueta para los botones
        self.label_buttons = tk.Label(self.frame_buttons, text="Acciones:", font=("Arial", 12))
        self.label_buttons.grid(row=0, column=0, padx=10, pady=5, sticky="w")

        # Botón para agregar archivos
        self.add_button = tk.Button(self.frame_buttons, text="Agregar Archivos", command=self.add_files)
        self.add_button.grid(row=1, column=0, padx=10, pady=5)

        # Botón para fusionar los PDFs
        self.merge_button = tk.Button(self.frame_buttons, text="Fusionar PDFs", command=self.merge_pdfs)
        self.merge_button.grid(row=1, column=1, padx=10, pady=5)

        # Frame para el Listbox y botones de reordenar
        self.frame_list = tk.Frame(self.root)
        self.frame_list.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")

        # Etiqueta para la lista de archivos
        self.label_list = tk.Label(self.frame_list, text="Archivos PDF a fusionar:", font=("Arial", 12))
        self.label_list.grid(row=0, column=0, padx=10, pady=5, sticky="w")

        # Lista de archivos (con capacidad de arrastrar y soltar)
        self.file_listbox = tk.Listbox(self.frame_list, selectmode=tk.SINGLE, width=50, height=10)
        self.file_listbox.grid(row=1, column=0, padx=10, pady=5)

        # Botones para mover los archivos hacia arriba y abajo
        self.move_up_button = tk.Button(self.frame_list, text="↑", command=self.move_up)
        self.move_up_button.grid(row=1, column=1, padx=10, pady=5, sticky="n")

        self.move_down_button = tk.Button(self.frame_list, text="↓", command=self.move_down)
        self.move_down_button.grid(row=1, column=1, padx=10, pady=5, sticky="s")

        # Configurar drag-and-drop
        self.file_listbox.drop_target_register(DND_FILES)
        self.file_listbox.dnd_bind('<<Drop>>', self.on_drop)

    def add_files(self):
        """Agrega archivos manualmente mediante un diálogo de archivo"""
        file_paths = filedialog.askopenfilenames(filetypes=[("Archivos PDF", "*.pdf")])
        
        if file_paths:
            for file_path in file_paths:
                if file_path not in self.files:
                    self.files.append(file_path)
                    self.file_listbox.insert(tk.END, file_path)
                else:
                    messagebox.showwarning("Advertencia", f"El archivo {file_path} ya está en la lista.")

    def on_drop(self, event):
        """Añade archivos arrastrados a la lista"""
        file_path = event.data
        if file_path.endswith(".pdf"):
            if file_path not in self.files:
                self.files.append(file_path)
                self.file_listbox.insert(tk.END, file_path)
            else:
                messagebox.showwarning("Advertencia", f"El archivo {file_path} ya está en la lista.")
        else:
            messagebox.showwarning("Advertencia", "Solo puedes agregar archivos PDF.")

    def move_up(self):
        """Mueve el archivo seleccionado hacia arriba"""
        selected_index = self.file_listbox.curselection()
        if selected_index:
            selected_index = selected_index[0]
            if selected_index > 0:
                # Intercambiar los elementos en la lista interna y en el Listbox
                self.files[selected_index], self.files[selected_index - 1] = self.files[selected_index - 1], self.files[selected_index]
                self.update_listbox()

    def move_down(self):
        """Mueve el archivo seleccionado hacia abajo"""
        selected_index = self.file_listbox.curselection()
        if selected_index:
            selected_index = selected_index[0]
            if selected_index < len(self.files) - 1:
                # Intercambiar los elementos en la lista interna y en el Listbox
                self.files[selected_index], self.files[selected_index + 1] = self.files[selected_index + 1], self.files[selected_index]
                self.update_listbox()

    def update_listbox(self):
        """Actualiza la visualización del Listbox con el nuevo orden"""
        self.file_listbox.delete(0, tk.END)
        for file_path in self.files:
            self.file_listbox.insert(tk.END, file_path)

    def merge_pdfs(self):
        """Fusiona los PDFs en un único archivo"""
        if not self.files:
            messagebox.showwarning("Advertencia", "No hay archivos PDF para fusionar.")
            return
        
        # Crear un objeto PDF merger
        pdf_merger = PyPDF2.PdfMerger()
        
        # Agregar los archivos PDF en el orden que están en la lista
        for file in self.files:
            pdf_merger.append(file)
        
        # Guardar el archivo final
        save_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
        
        if save_path:
            pdf_merger.write(save_path)
            messagebox.showinfo("Éxito", f"PDF fusionado guardado en {save_path}")
            self.files.clear()
            self.file_listbox.delete(0, tk.END)

# Inicializar la ventana de la aplicación
if __name__ == "__main__":
    root = TkinterDnD.Tk()  # Usamos TkinterDnD para soportar drag-and-drop
    app = PDFMergerApp(root)
    root.mainloop()
```

### Mejoras implementadas:

#### 1. **Reordenar los archivos:**
   - Añadí los botones de mover hacia arriba (`↑`) y hacia abajo (`↓`) para permitir reorganizar los archivos dentro del `Listbox`.
   - Estos botones funcionan sobre el archivo que esté seleccionado en la lista. El orden se actualiza automáticamente cuando mueves los archivos.

#### 2. **Interfaz más amigable:**
   - Se usó `grid` para organizar los elementos de la interfaz de forma más estructurada.
   - Añadí una etiqueta que indica "Acciones" para los botones y otra para la lista de archivos PDF.
   - Los botones de mover y las etiquetas están mejor organizados para hacer que la interfaz se vea más profesional.

### Cómo usar la aplicación:
1. **Agregar archivos**: Usa el botón "Agregar Archivos" para seleccionar archivos PDF o arrástralos directamente a la lista.
2. **Reordenar archivos**: Selecciona un archivo de la lista y usa los botones "↑" o "↓" para cambiar su posición.
3. **Fusionar archivos**: Una vez que tengas los archivos en el orden correcto, presiona "Fusionar PDFs" para combinarlos en un solo archivo PDF.

### Requisitos:
Asegúrate de tener las librerías necesarias instaladas:

```bash
pip install PyPDF2 tkinterdnd2
```

¡Listo! Ahora tienes una interfaz gráfica mejorada y funcionalidad de reordenación de los archivos. ¿Te gustaría agregar alguna otra característica o mejorar algo más?

Para convertir un programa de Python que utiliza `tkinter`, `PyPDF2`, y `tkinterdnd2` en un ejecutable `.exe` para Windows 10, hay varias herramientas confiables. La más común y recomendada es **PyInstaller**, pero también existen otras opciones. Aquí te explico las principales:

---

### ✅ **1. PyInstaller (recomendado)**

**Ventajas:**
- Muy usado, bien documentado.
- Compatible con `tkinter` y `tkinterdnd2`.
- Empaqueta todo en un solo `.exe` si lo deseas.

**Pasos básicos:**

1. Instala PyInstaller:

   ```bash
   pip install pyinstaller
   ```

2. Ve al directorio del script y ejecuta:

   ```bash
   pyinstaller --onefile --noconsole tu_script.py
   ```

   - `--onefile`: crea un único `.exe`.
   - `--noconsole`: útil si tu app es gráfica y no necesita terminal.

3. El `.exe` estará en la carpeta `dist/`.

⚠️ **Nota**: `tkinterdnd2` a veces requiere incluir archivos adicionales. Si el `.exe` no funciona correctamente, intenta sin `--onefile` o agrega recursos manualmente usando el parámetro `--add-data`.

---

### ✅ **2. cx_Freeze**

Otra alternativa robusta, aunque un poco menos sencilla que PyInstaller.

```bash
pip install cx_Freeze
```

Luego debes crear un archivo `setup.py` para configurar la build.

---

### ✅ **3. auto-py-to-exe (interfaz gráfica para PyInstaller)**

Esencialmente es PyInstaller con GUI, ideal si no querés tocar la terminal mucho.

1. Instalar:

   ```bash
   pip install auto-py-to-exe
   ```

2. Ejecutar:

   ```bash
   auto-py-to-exe
   ```

3. Desde la interfaz podés seleccionar opciones como:
   - `.onefile` o carpeta
   - GUI app o consola
   - Archivos adicionales

---

### 📌 Consideraciones Especiales para `tkinterdnd2`

- A veces `tkinterdnd2` requiere incluir manualmente sus archivos `.tcl`.
- Si el `.exe` falla al iniciar la ventana o el arrastrar y soltar, probá ejecutar el build sin `--onefile` o copiá la carpeta `tkdnd` al lado del `.exe`.

---

### ¿Querés que te ayude a generar el comando exacto según tu estructura de carpetas y nombre del script?

Si me pasás:
- El nombre del archivo `.py`
- Si usás imágenes o recursos externos
- Si querés `.exe` con consola o sin consola

Te armo el comando ideal.